In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from keras.src.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPool2D, Embedding, LSTM
from nltk.corpus import stopwords
import re
import string
import nltk
from nltk.corpus import stopwords


nltk.download('stopwords')
df = pd.read_csv('legal_text_classification.csv', sep=',', usecols=["case_outcome", "case_text"])
outcome_mapping = {'cited': 0, 'applied': 1}

# Create a new column 'outcome_numeric' based on the mapping
df['outcome_numeric'] = df['case_outcome'].map(outcome_mapping)
#df.head() 


def clean_the_text(text):
  stop_words = stopwords.words("english")
  text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
  text = text.lower()
  text = ' '.join([word for word in text.split(' ') if word not in stop_words])
  #print(text)
  return text
df['case_text'] = df['case_text'].apply(lambda x: str(x) if isinstance(x, (str, float)) else '').apply(clean_the_text)
#df['case_outcome'] = df['case_outcome'].apply(lambda x: str(x) if isinstance(x, (str, float)) else '').apply(clean_the_text)


X = df['case_text']
y = df['outcome_numeric']
#y = y.astype(float)
print("head")
print(df['case_outcome'][0:5])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12345)

print("info")
df.info()

token = Tokenizer(lower=False)
token.fit_on_texts(X_train)
X_train = token.texts_to_sequences(X_train)
X_test = token.texts_to_sequences(X_test)

array = []
for i_train in X_train:
    array.append(len(i_train))
maximum_len = int(np.ceil(np.mean(array))) 

X_train = pad_sequences(X_train, maxlen=maximum_len)
X_test = pad_sequences(X_test, maxlen=maximum_len)
total_words = len(token.word_index) + 1
model = keras.Sequential()
model.add(Embedding(total_words, 32, input_length= maxlen))
model.add(LSTM(10, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

[nltk_data] Downloading package stopwords to /home/maksym/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


head
0    cited
1    cited
2    cited
3    cited
4    cited
Name: case_outcome, dtype: object
info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24985 entries, 0 to 24984
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   case_outcome     24985 non-null  object 
 1   case_text        24985 non-null  object 
 2   outcome_numeric  14667 non-null  float64
dtypes: float64(1), object(2)
memory usage: 585.7+ KB
Epoch 1/10
625/625 [==============================] - 60s 93ms/step - loss: nan - accuracy: 0.4866
Epoch 2/10
625/625 [==============================] - 50s 80ms/step - loss: nan - accuracy: 0.4868
Epoch 3/10
625/625 [==============================] - 50s 81ms/step - loss: nan - accuracy: 0.4868
Epoch 4/10
625/625 [==============================] - 56s 90ms/step - loss: nan - accuracy: 0.4868
Epoch 5/10
625/625 [==============================] - 58s 93ms/step - loss: nan - accuracy: 0.4868
Epoch 6/10
625